In [14]:
import pandas as pd

In [15]:
nasa_df=pd.read_csv('../Datasets/wildfire_df.csv')

C:\Users\Olan\AppData\Local\Temp\ipykernel_30660\1027712823.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  nasa_df=pd.read_csv('../Datasets/wildfire_df.csv')


In [16]:
wildfire_df=nasa_df[(nasa_df['state']=='California') & (nasa_df['acq_date']>'2020-01-01') & (nasa_df['latitude']<=36)].copy()
# southern california
#                    (nasa_df['latitude']>=32) & (nasa_df['latitude']<=34.5) & 
#                     (nasa_df['longitude']>=-118) & (nasa_df['longitude']<=-116) 

In [17]:
# Dropping rows where same day and same time there were multiple rows
wildfire_df.dropna(subset=['acq_time','acq_date'],inplace=True)

In [18]:
# fixing time column  properly
wildfire_df['acq_time'] = wildfire_df['acq_time'].astype(str).str.zfill(4)  # Ensure leading zeros for hours < 10
wildfire_df['acq_time'] = pd.to_datetime(wildfire_df['acq_time'], format='%H%M').dt.time

In [19]:
wildfire_df['acq_time']= pd.to_datetime(wildfire_df['acq_time'], format='%H:%M:%S')
wildfire_df['nearest_hour']=wildfire_df['acq_time'].dt.round('H').dt.time
wildfire_df['acq_time']=wildfire_df['acq_time'].dt.time

In [95]:
# wildfire_df.to_csv('../Datasets/filtered_wildfire_df.csv')
wildfire_df.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,state,nearest_hour
5276400,33.77762,-118.22819,297.45,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,283.34,1.14,N,0.0,California,09:00:00
5276401,33.77818,-118.23756,298.90,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,285.94,1.01,N,0.0,California,09:00:00
5276402,33.81285,-118.23906,297.82,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,285.95,0.99,N,0.0,California,09:00:00
5276403,33.81887,-118.24325,296.60,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,285.98,1.08,N,2.0,California,09:00:00
5276404,33.81951,-118.24489,298.51,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,286.40,0.92,N,2.0,California,09:00:00


In [21]:
# !pip3 install openmeteo_requests requests_cache retry_requests

In [22]:
import openmeteo_requests

import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [23]:
url = "https://archive-api.open-meteo.com/v1/archive"

In [309]:
test_data=wildfire_df.copy()

In [301]:
import time

In [270]:
# hourly_dataframe = pd.DataFrame()

In [302]:
count=0
for index, row in test_data.iterrows():
    if(count%50==0):
        print(count,"rows done!!")
    params = {
        "latitude": row["latitude"],
        "longitude": row["longitude"],
        "start_date": row['acq_date'],
        "end_date": row['acq_date'],
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", 
                   "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", 
                   "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", 
                   "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", 
                   "wind_direction_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", 
                   "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", 
                   "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
        "timezone": "America/Los_Angeles"
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_soil_temperature_0_to_7cm = hourly.Variables(22).ValuesAsNumpy()
    hourly_soil_temperature_7_to_28cm = hourly.Variables(23).ValuesAsNumpy()
    hourly_soil_temperature_28_to_100cm = hourly.Variables(24).ValuesAsNumpy()
    hourly_soil_temperature_100_to_255cm = hourly.Variables(25).ValuesAsNumpy()
    hourly_soil_moisture_0_to_7cm = hourly.Variables(26).ValuesAsNumpy()
    hourly_soil_moisture_7_to_28cm = hourly.Variables(27).ValuesAsNumpy()
    hourly_soil_moisture_28_to_100cm = hourly.Variables(28).ValuesAsNumpy()
    hourly_soil_moisture_100_to_255cm = hourly.Variables(29).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["latitude"] = row["latitude"]
    hourly_data["longitude"] = row["longitude"]
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
    hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
    hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
    hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
    hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
    hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
    hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
    hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm
    hourly_dataframe=pd.concat([hourly_dataframe,pd.DataFrame(data = hourly_data)]) 
    time.sleep(1)
    count+=1

30650 rows done!!
30700 rows done!!
30750 rows done!!
30800 rows done!!
30850 rows done!!
30900 rows done!!
30950 rows done!!
31000 rows done!!


In [304]:
hourly_dataframe.to_csv('../Datasets/hourly_dataframe.csv',index=False)

In [307]:
# hourly_weather_df=pd.read_csv('../Datasets/hourly_dataframe.csv')